# Bike Rebalancing Optimization Simulation

In [50]:
import SimFunctions
import SimRNG 
import SimClasses
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.stats as stats
import xlrd

In [51]:
# Read the data file we got through data cleaning. 
wb = xlrd.open_workbook('BikeRentCount.xlsx')
# The file consists of five sheets and each sheet contains data for one station.
sheets = wb.sheet_names()
MaxRentingRate=[]
RentingRate=[]
for i in range(len(sheets)):
    data = pd.read_excel('BikeRentCount.xlsx', sheet_name=sheets[i])
    #Calculate the average number of rented bikes for each twenty minutes of each station.
    rentingRates = data.mean()
    RentingRate.append(rentingRates)
    # Find the maximum of these rates that would be used for function NSPP
    maxRentingRate = max(rentingRates)
    MaxRentingRate.append(maxRentingRate)

In [39]:
c = ["Bay St / St. Joseph St", "Union Station", "College St / Major St", "Queens Quay / Yonge St", "Madison Ave / Bloor St W"]
Rent = pd.DataFrame(columns = c, index = ["7:00-7:20", "7:20-7:40", "7:40-8:00", "8:20-8:40", "8:40-9:00", 
                                          "9:00-9:20", "9:20-9:40", "9:40-10:00", "MaxRentingRate"])

In [43]:
for i in range(0,5,1):
    Rent[c[i]] = RentingRate[i]
Rent.loc['MaxRentingRate'] = MaxRentingRate
Rent

,Bay St / St. Joseph St,Union Station,College St / Major St,Queens Quay / Yonge St,Madison Ave / Bloor St W
7:00-7:20,0.336957,1.869565,0.250000,0.173913,0.152174
7:20-7:40,0.945652,4.967391,0.608696,0.206522,0.184783
7:40-8:00,1.271739,7.923913,0.304348,0.282609,0.282609
8:20-8:40,1.652174,6.086957,0.521739,0.945652,0.695652
8:40-9:00,1.630435,4.717391,1.184783,1.141304,1.239130
9:00-9:20,1.173913,3.478261,0.923913,0.847826,1.163043
9:20-9:40,1.184783,3.217391,0.913043,0.597826,0.532609
9:40-10:00,0.728261,1.706522,0.597826,1.010870,0.500000
MaxRentingRate,1.652174,7.923913,1.184783,1.141304,1.239130


In [5]:
# Similar data processing for 'BikeReturnCount.xlsx'
wb = xlrd.open_workbook('BikeReturnCount.xlsx')
sheets = wb.sheet_names()
MaxReturnRate=[]
ReturnRate=[]
for i in range(len(sheets)):
    data = pd.read_excel('BikeReturnCount.xlsx', sheet_name=sheets[i])
    returnRates = data.mean()
    ReturnRate.append(returnRates)
    maxReturnRate = max(returnRates)
    MaxReturnRate.append(maxReturnRate)

In [44]:
Return = pd.DataFrame(columns = c, index = ["7:00-7:20", "7:20-7:40", "7:40-8:00", "8:20-8:40", "8:40-9:00", 
                                            "9:00-9:20", "9:20-9:40", "9:40-10:00", "MaxReturnRate"])
for i in range(0,5,1):
    Return[c[i]] = ReturnRate[i]
Return.loc['MaxReturnRate'] = MaxReturnRate
Return

,Bay St / St. Joseph St,Union Station,College St / Major St,Queens Quay / Yonge St,Madison Ave / Bloor St W
7:00-7:20,0.054348,1.467391,0.043478,0.054348,0.097826
7:20-7:40,0.293478,1.739130,0.086957,0.217391,0.097826
7:40-8:00,0.304348,2.413043,0.141304,0.413043,0.250000
8:20-8:40,0.826087,4.206522,0.413043,1.032609,0.195652
8:40-9:00,0.880435,3.869565,1.065217,1.923913,0.641304
9:00-9:20,0.902174,4.141304,0.760870,1.000000,0.195652
9:20-9:40,0.489130,3.021739,0.619565,0.771739,0.184783
9:40-10:00,0.380435,1.652174,0.815217,0.858696,0.206522
MaxReturnRate,0.902174,4.206522,1.065217,1.923913,0.641304


## Simulation constants

In [6]:
order_threshold = 5.0 #schedule the "rebalancing" once the number of bikes is less than the threshold
order_up_to = 10.0 # the number of bikes in the station would reach 5 after rebalancing
delivery_delay = 20 # it takes 20 minutes for bikes to be transported to the station for bike rebalancing
replications = 10 #number of simulation runs
operation_cost = 2 # CAD per bike for operation
oil_gas = 3 # CAD per 1 refillment
service_fee = 3.75 # CAD per bike per ride
PENALTY = 3.75 # CAD for cost of loss of business oportunity
loss_profit = 0.01 # CAD per bike per minute for loss of business opportunity
RunLength = 180 # 3 hours

## Main Simulation Process

In [7]:
def Rental(**kwargs):
    global revenue, penalty, Loss_profit
    #Check which station the rental event occurs in
    station_id = kwargs['stationID']
    Pickup_queue = kwargs["pickup_queue"]
    Dropoff_queue = kwargs["dropoff_queue"]
    #Use NSPP to schedule the next bike rental event for current station
    interarrival = NSPP(station_id,MaxRentingRate,RentingRate)
    SimFunctions.Schedule(Calendar,"Rent a bike",interarrival,stationID = station_id,pickup_queue = TheQueues[station_id],dropoff_queue = TheQueues[station_id+5])

    # Checks if there are people waiting to put the bikes back.
    if Dropoff_queue.NumQueue() > 0:
        DepartingCustomer = Dropoff_queue.Remove()
        Waiting_time = SimClasses.Clock - DepartingCustomer.CreateTime
        Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
        # If customer has waited too long to return the bike, refund is given.
        # The customer gets the bike from the people waiting to put the bikes back.
        if Waiting_time >5:
            penalty += service_fee
            
    #If no one is waiting to put the bikes back,
    #check if there are bikes available and update number of bikes for the station.
    else:
        if Num_bikes[station_id]>0:
            Num_bikes[station_id]-=1
            # Customer pays to rent bike.
            revenue += service_fee
        #No bikes available so the customer begins waiting for bikes to become available.
        else:
            Customer = SimClasses.Entity()
            Pickup_queue.Add(Customer)
            
    #Schedule the bike rebalancing event once the number of bikes is less than the threshold 
    """
    There are two conditions to be met for the rebalancing operation. Schedule the rebalancing operation 
    once the number of bikes is less than the threshold and the previous rebalance operation has ended. 
    """
    if Num_bikes[station_id] <= order_threshold and quantity[station_id] == 0:
        quantity[station_id] = order_up_to - Num_bikes[station_id]
        SimFunctions.Schedule(Calendar,"rebalancing",delivery_delay,stationID=station_id,pickup_queue = TheQueues[station_id],dropoff_queue = TheQueues[station_id+5],num_ordered=quantity[station_id], signal= -1)

In [8]:
def RideEnd(**kwargs):
    global  revenue, penalty, Loss_profit
    station_id = kwargs['stationID']
    Dropoff_queue = kwargs['dropoff_queue']
    Pickup_queue = kwargs['pickup_queue']
     ##Use NSPP to schedule the next end of ride for current station
    interarrival = NSPP(station_id,MaxReturnRate,ReturnRate)
    SimFunctions.Schedule(Calendar,"Return a bike",interarrival,stationID = station_id,pickup_queue = TheQueues[station_id],dropoff_queue = TheQueues[station_id+5])
    
    # We assume not every customer will wait for a bike and eventually take a bike.Customers leave after 5 mins.
    while(True):
        # Check if customers are waiting.
        if Pickup_queue.NumQueue() == 0:
            #If no one is waiting to pick up the bike,check if there are empty racks to keep the bike.   
            if Num_bikes[station_id] < Num_docks[station_id]:
                # Customer returns the bike to the rack.
                Num_bikes[station_id] += 1
            # No empty racks. The customer begins waiting in queue.
            else:
                Customer = SimClasses.Entity()
                Dropoff_queue.Add(Customer)
            break
        #If there are customers waiting to pick up the bike,
        #check if the customer has left due to waiting too long to pick up the bike.
        DepartingCustomer = Pickup_queue.Remove()
        Waiting_time = SimClasses.Clock - DepartingCustomer.CreateTime
        Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
        if Waiting_time < 5:
            # Next waiting customer gets a bike from the customer who has finished the ride.
            # Break the while loop.
            break
        else:
            # We lose a customer because the customer has waited too long
            penalty += PENALTY
            
    if Dropoff_queue.NumQueue() >= order_threshold and quantity[station_id] == 0:
        quantity[station_id] = Dropoff_queue.NumQueue()
        SimFunctions.Schedule(Calendar,"rebalancing",delivery_delay,stationID=station_id,pickup_queue = TheQueues[station_id],dropoff_queue = TheQueues[station_id+5],num_ordered=quantity[station_id], signal= 1)

In [9]:
def rebalancing(**kwargs):
    global revenue , cost, Loss_profit,penalty
    station_id = kwargs['stationID']
    Dropoff_queue = kwargs['dropoff_queue']
    Pickup_queue = kwargs['pickup_queue']
    #The number of bikes needed for bike rebalancing
    Num_ordered = kwargs['num_ordered']
    Signal = kwargs['signal']
    #Cost due to bike rebalanccing
    cost += (Num_ordered * operation_cost) + oil_gas 
    # Signal is -1 means the station is almost out of bikes for customers renting bikes, 
    # then the center will carry bikes to the station 
    if Signal == -1:
        #The number of bikes changes after the rebalancing operation
        Num_bikes[station_id] += Num_ordered
        # We assume not every customer will wait for a bike and eventually take a bike..Customers leave after 5 mins.
        while(True):
            # Check whether customers are waiting in the pick_up queue and whether the ordered number of bikes is in short supply.
            if (Pickup_queue.NumQueue() == 0.0) | (Num_bikes[station_id] == 0.0):
                break

            DepartingCustomer = Pickup_queue.Remove()
            Waiting_time = SimClasses.Clock - DepartingCustomer.CreateTime
            Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
            if Waiting_time < 5:
                # Next waiting customer gets a bike
                 Num_bikes[station_id] -= 1
            else:
                # We lose a customer
                penalty += PENALTY
    # Signal is +1 means the station is out of racks for customers returning bikes, 
    # then the center will carry bikes back to the center             
    else:
        while(True):
            # Check if customers are waiting in the Dropoff queue
            if (Dropoff_queue.NumQueue() == 0.0)| (Num_ordered==0.0):
                break

            DepartingCustomer = Dropoff_queue.Remove()
            Num_ordered-=1
            Waiting_time = SimClasses.Clock - DepartingCustomer.CreateTime
            Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
            if Waiting_time > 5:
                # If customer has waited too long to return the bike, refund is given.
                 penalty += service_fee
        Num_bikes[station_id] -= Num_ordered
    #Set the num_oredered to zero so the current rebalancing operation is over.
    quantity[station_id] = 0

In [10]:
# Prevent index out of range
def PW_ArrRate(p,t,Rates):
    hour = int(t/20)
    if hour <= 8:
        return Rates[p][hour]
    else:
        return Rates[p][-1]

In [11]:
def NSPP(q,MaxRate,Rate):
    PossibleArrival = SimClasses.Clock + SimRNG.Expon(1/(MaxRate[q]/20), 1)
    while SimRNG.Uniform(0, 1, 1) >= PW_ArrRate(q,PossibleArrival,Rate)/(MaxRate[q]):
        PossibleArrival = PossibleArrival + SimRNG.Expon(1/(MaxRate[q]/20), 1)
    nspp = PossibleArrival - SimClasses.Clock
    return nspp

In [12]:
# Get all trial solution 
trial_solution = []
for a in range(5,15):
    for b in range(5,27):
        for c in range(5,11):
            for d in range(5,15):
                for e in range(5,15):
                    # Initial number of bikes available in the system.
                    if (a+b+c+d+e)==70:
                        trial_solution.append([a,b,c,d,e])
len(trial_solution)

480

In [52]:
Calendar = SimClasses.EventCalendar()
Wait = SimClasses.DTStat()

Pickup_Queue1 = SimClasses.FIFOQueue()
Pickup_Queue2 = SimClasses.FIFOQueue()
Pickup_Queue3 = SimClasses.FIFOQueue()
Pickup_Queue4 = SimClasses.FIFOQueue()
Pickup_Queue5 = SimClasses.FIFOQueue()

Dropoff_Queue1 = SimClasses.FIFOQueue()
Dropoff_Queue2 = SimClasses.FIFOQueue()
Dropoff_Queue3 = SimClasses.FIFOQueue()
Dropoff_Queue4 = SimClasses.FIFOQueue()
Dropoff_Queue5 = SimClasses.FIFOQueue()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

TheQueues.append(Pickup_Queue1)
TheQueues.append(Pickup_Queue2)
TheQueues.append(Pickup_Queue3)
TheQueues.append(Pickup_Queue4)
TheQueues.append(Pickup_Queue5)
TheQueues.append(Dropoff_Queue1)
TheQueues.append(Dropoff_Queue2)
TheQueues.append(Dropoff_Queue3)
TheQueues.append(Dropoff_Queue4)
TheQueues.append(Dropoff_Queue5)

TheDTStats.append(Wait)

AllWaitMean = np.zeros((replications,len(trial_solution)))

ZSimRNG = SimRNG.InitializeRNSeed()

#Create an 2d array to record the balance of each trial solution in each simulation run
output = np.zeros((replications,len(trial_solution)))
for k in range(replications):
    for j in range(len(trial_solution)):
        Num_bikes=[]
        initial_Num_bikes=[]
        #Apply the our model to each trial solution
        for i in range(0,5,1):
            Num_bikes.append(trial_solution[j][i])
            initial_Num_bikes.append(trial_solution[j][i])
        # Number of bike racks (total) at each station. This is the maximum parking
        # capacity for every station.
        Num_docks=[15,27,11,15,15]
        #initial number of ordered bikes for rebalancing
        quantity=[0,0,0,0,0]
        revenue = 0
        cost = 0
        penalty = 0
        Loss_profit = 0

        SimClasses.Clock = 0.0
        SimFunctions.SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)
        for i in range(0,5,1):
            #Use NSPP to schedule the first arrival for each station.
            #Initialize queues at each station.
            SimFunctions.Schedule(Calendar,"Rent a bike",NSPP(i,MaxRentingRate,RentingRate),stationID = i,pickup_queue = TheQueues[i],dropoff_queue = TheQueues[i+5])
            SimFunctions.Schedule(Calendar,"Return a bike",NSPP(i,MaxReturnRate,ReturnRate),stationID = i,pickup_queue = TheQueues[i],dropoff_queue = TheQueues[i+5]) 
        SimFunctions.Schedule(Calendar,"EndSimulation",RunLength)
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Rent a bike":
            Rental(**NextEvent.kwargs)
        elif NextEvent.EventType == "Return a bike":
            RideEnd(**NextEvent.kwargs)
        elif NextEvent.EventType == "rebalancing":
            rebalancing(**NextEvent.kwargs)

        while NextEvent.EventType != "EndSimulation":
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            if NextEvent.EventType == "Rent a bike":
                Rental(**NextEvent.kwargs)
            elif NextEvent.EventType == "Return a bike":
                RideEnd(**NextEvent.kwargs)
            elif NextEvent.EventType == "rebalancing":
                rebalancing(**NextEvent.kwargs)
        repositioning_bikes= (abs(np.array(Num_bikes)-np.array(initial_Num_bikes)).sum())
        cost += (repositioning_bikes * operation_cost) + oil_gas 
        balance = revenue - cost - penalty - Loss_profit
        output[k][j]= float(balance)
        AllWaitMean[k][j] = Wait.Mean()

In [53]:
output

array([[156.  , 144.  , 134.75, ...,  82.25, 134.  , 146.  ],
       [103.5 , 113.  , 187.  , ..., 128.5 , 192.  ,  42.75],
       [180.5 , 173.5 , 136.25, ...,  85.75, 161.5 , 142.75],
       ...,
       [133.75, 166.75, 219.25, ..., 148.75, 200.  ,  53.  ],
       [112.75, 222.75, 209.5 , ..., 165.25, 149.25, 144.5 ],
       [188.  , 152.25, 193.25, ..., 177.5 , 156.5 , 104.75]])

In [15]:
AllWaitMean

array([[79.68867137, 14.37337583,  0.        , ..., 14.76020076,
        20.2248651 ,  7.22097653],
       [13.41130522, 32.41174321,  4.73163264, ...,  3.62623428,
        24.03712014, 52.41381169],
       [20.38176342,  0.        ,  9.56696173, ..., 20.89614167,
         0.        , 27.01509396],
       ...,
       [ 0.        , 15.45577394, 13.45338826, ..., 31.60409876,
        10.88522418, 24.04242332],
       [17.88555415, 15.63596133, 13.64442013, ...,  5.99842879,
        21.95294725, 23.19763369],
       [18.28015825, 13.72281172,  7.12195601, ...,  0.        ,
         3.97859463, 28.81891739]])

In [54]:
#Get the the expectation by calculating the mean over all simulation runs for each trial solution
Estimated_Expected_waittime=np.mean(AllWaitMean,axis=0)
#Get the best solution to minimize customers waiting time. 
MinWaitingTime = trial_solution[np.argmin(Estimated_Expected_waittime)]

In [55]:
#Get the the expectation by calculating the mean over all simulation runs for each trial solution
Estimated_Expected_balance=np.mean(output,axis=0)
#Get the best solution to maximize the balance.
MaxBalance = trial_solution[np.argmax(Estimated_Expected_balance)]

In [56]:
result = pd.DataFrame(columns = ["MinWaitingTime", "MaxBalance"], index = ["Bay St / St. Joseph St", "Union Station", 
                                "College St / Major St", "Queens Quay / Yonge St", "Madison Ave / Bloor St W"])
result['MinWaitingTime'] = MinWaitingTime
result['MaxBalance'] = MaxBalance
result

,MinWaitingTime,MaxBalance
Bay St / St. Joseph St,13,13
Union Station,24,26
College St / Major St,7,8
Queens Quay / Yonge St,12,10
Madison Ave / Bloor St W,14,13
